In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""  

import numpy as np
import tensorflow.compat.v1 as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

from data_factory.takedata import LoadData
from data_factory.dataloader import BatchLoader
from model_core.state_model.cluster import ClusterStateRecognition
from model_core.models.EvoNet.evonet_tsc import EvoNet_TSC
from model_core.config import ModelParam

def demo_test():
    params = ModelParam()
    params.data_name = 'djia30'
    datainfos = {
        'djia30':    [50,  5, 4, 3.0],
        # 'webtraffic':[12, 30, 1, 3.0],
    }
    params.his_len, params.segment_len, params.segment_dim, params.pos_weight = datainfos[params.data_name]
    params.node_dim = 2 * params.segment_len * params.segment_dim

    loader = LoadData()
    loader.set_configuration(params)
    x_train, y_train, x_test, y_test = loader.fetch_data()
    print(f"Loaded test set: {x_test.shape[0]} samples of shape {x_test.shape[1:]}")

    DEMO_SAMPLES = 1500
    x_demo = x_test[:DEMO_SAMPLES]
    y_demo = y_test[:DEMO_SAMPLES]

    state_model = ClusterStateRecognition()
    state_model.set_configuration(params)
    state_model.build_model()
    state_model.restore(params.model_save_path)
    demo_prob, demo_patterns = state_model.predict(x_demo)

    BATCH_SIZE = 128
    batch_loader = BatchLoader(batch_size=BATCH_SIZE)
    batch_loader.load_data(
        x=x_demo,
        y=y_demo,
        prob=demo_prob,
        patterns=demo_patterns,
        shuffle=False
    )

    tf.disable_v2_behavior()
    tf.reset_default_graph()
    model = EvoNet_TSC()
    model.set_configuration(params)
    model.build_model(is_training=False)

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    model.restore(params.model_save_path, sess)

    preds, pred_probas = model.predict(sess, dataloader=batch_loader)
    true_labels = y_demo[:, -1]

    acc  = accuracy_score(true_labels, preds)
    prec = precision_score(true_labels, preds, zero_division=0)
    rec  = recall_score(true_labels, preds, zero_division=0)
    f1   = f1_score(true_labels, preds, zero_division=0)
    auc  = roc_auc_score(true_labels, pred_probas[:, 1])

    print(f"\nDemo on first {DEMO_SAMPLES} test samples:")
    print(f"  Accuracy : {acc:.4f}")
    print(f"  Precision: {prec:.4f}")
    print(f"  Recall   : {rec:.4f}")
    print(f"  F1 Score : {f1:.4f}")
    print(f"  AUC      : {auc:.4f}")

    sess.close()

demo_test()

load djia30 data
Loaded test set: 2790 samples of shape (50, 5, 4)
INFO:tensorflow:Restoring parameters from ./model/ETNet_djia30_30/model

Demo on first 1500 test samples:
  Accuracy : 0.6833
  Precision: 0.3965
  Recall   : 0.5501
  F1 Score : 0.4608
  AUC      : 0.6917
